# Euler's Method for Initial-Value Problems

## Overview, Objectives, and Key Terms
 
In this lecture, we review the basics of *first-order*, *ordinary differential equations* and their role in *initial value problems*.  Some of the basic solution techniques 

revisit the tools provided by SymPy and apply them to solution of *ordinary differential equations* (ODEs).  Engineering is full of ODEs, since they are fundamental to the language of balance relationships found in heat transfer, nuclear reactor physics, control systems, and other areas.  The goal here is *not* to present the theory of ODEs, nor is it to present the various systematic ways be which one can determine solutions to ODEs.  Rather, the focus is placed squarely on defining ODEs symbolically, solving them symbolically, and, *importantly*, applying initial or boundary conditions to those  symbolic solutions and solving the resulting *algebraic* equations for any undetermined coefficients.  In other words, the focus is on the *problem solving* process.


### Objectives

By the end of this lesson, you should be able to

- Define first-order IVPs given engineering input (e.g., a description of some system)
- Solve first-order IVPs analytically (prior knowledge in part)
- Solve first-order IVPs numerically using the forward-Euler and backward-Euler methods.

### Prerequisites

You should already have a basic comprehension of ODEs, especially IVPs, at the level covered in MATH 340 (now a pre-requisite for this course).  That includes deducing general solutions, applying initial conditions, and determining complete solutions.  In fact, you should also know something about Laplace transforms and Euler's method, the latter technique which is the focus of this lesson.  Specifically, you should, given a day or two of review, be able to sit down and complete a good chunk of an exam like [this one](https://www.math.ksu.edu/courses/exam-archive/math240/Math240fxS17.pdf) at a passing level.  If that is *not* the case, see an instructor in office hours and/or begin to review the [MATH 340 textbook](https://www.math.ksu.edu/math240/book/).


### Key Terms

- differential equation
- ordinary
- linear
- ODE
- initial value problem
- boundary value problem

## Differential Equations

A *differential equation* is any equation in which an unknown function $f$ appears with one of more of its derivatives.  When $f$ is a function of only one variable, the derivatives are *ordinary* (as opposed to *partial*), and the equation is an *ordinary differential equation* (ODE).  In compact notation, such an equation may be written
\begin{equation}
    \mathcal{L}f(x) = q(x) \, ,
\end{equation}
where $\mathcal{L}$ is a differential operator.  If $\mathcal{L}$ contains only *linear combinations* of $f(x)$ and its derivates, then $\mathcal{L}$ is a *linear operator*, and the equation is a *linear* ODE.  If $q(x) = 0$, the equation is *homogeneous* and is otherwise *inhomogeneous*.  If the highest derivative of $f(x)$ present in $\mathcal{L}f$ is $n$ (i.e., $d^n f/dx^n$), then the equation is called an *n*th-order equation.

Examples of linear operators include $\mathcal{L}f = \frac{df}{dx}$ and $\mathcal{L}f = a(x) \frac{d^2f}{dx^2} + b(x)$, where $a(x)$ and $b(x)$ are arbitrary functions of $x$ (but not of $f(x)$).  For comparison, a nonlinear example is $\mathcal{L}f = \left ( \frac{df}{dx} \right )^2 - \sqrt{f(x)}$, which includes nonlinear functions of $f$ and its first derivative.

Our focus here and the next module is entirely on *ordinary differential equations* (both linear and nonlinear) and their solution by analytic and numerical techniques.

## Whence Comes $y'+py=q$?

Consider problems of the form

$$
    \frac{dy}{dt} + p(t)y(t) = q(t) \, , \qquad y(0) = y_0 \, .
$$

This is the canonical way to represent a **first-order**, **linear**, **initial-value problem** (IVP). It is first order because there is only a first derivative.  It is an initial-value problem because the unknown (here, $y(t)$) is specified at some "initial" time.  It is linear because $p(t)$ does not depend on $y(t)$.

A first-order IVP can be used to represent of a number of physical phenomena.  The first that comes to mind (to me, as a nuclear engineer) is [radioactive decay](https://en.wikipedia.org/wiki/Exponential_decay) with external production, usually written $N' = -\lambda N(t) + R(t)$, where $N(t)$ is the number of radioactive nuclei at time $t$, $\lambda$ is the decay constant (with units of inverse seconds), and $R(t)$ is the number of new nuclei born per second.  The time rate of change of $N(t)$ is a balance of those new nuclei from $R(t)$ against the number $-\lambda N(t)$ lost by decay.  For sufficiently long times, $dy/dt \to 0$, and $y = R/\lambda$, a *steady state* in which losses are exactly balanced by gains.

An equally good example is the case of an object of mass $m$ in [free fall through a viscuous medium](http://hyperphysics.phy-astr.gsu.edu/hbase/lindrg.html), e.g., the atmosphere.  If some object is held a distance above the earth and dropped, its acceleration at any point in time is related to the net forces acting on the body.  These forces include the gravitational force $F_g = -m g$ (i.e., downward) and the air resistance $F_r$.  A simple resistance model is $F_r = cv$, where $c$ is a (positive) constant and $v$ is the velocity directed toward Earth. Hence,
the downward force is 

$$
  F(t) = -cv(t) + m g \, .
$$

From Newton's second law, $F = ma$, so we have

$$
    ma(t) = m \frac{dv}{dt} = -cv(t)  + mg \, ,
$$

or

$$
    v' = -\frac{c}{m} v(t) + g \, .
$$

The positive $c/m$ has the same effect as a positive $\lambda$ in radioactive decay: the larger the velocity $v$, the larger the retardation force $cv$ (just like the larger the number of nuclei $N$, the larger the number $\lambda N$ decaying per unit time).  For the free-fall case, the steady-state condition $v'=0$ has a special name: *terminal velocity*.  

Lots of phenomena have this sort of behavior, this natural *negative feedback*, and such feedback can help us when we solve these problems numerically.  In those cases where the feedback is *positive*, we expect the number of nuclei to *grow* more and more (or, equivalently, for a falling object to keep accelerating toward earth).  The former happens when pythons (or neutrons) multiply in nature; thankfully, the [hail of northeastern Kansas](https://en.wikipedia.org/wiki/April_10%E2%80%9311,_2001_tornado_outbreak#Tri-state_hailstorm) does reach a terminal velocity.  Oh, and I'm not joking about pythons: combine the  [Malthusian model](https://en.wikipedia.org/wiki/Malthusian_growth_model) with the fact there is an artificial source term due to pet owners releasing their larger-than-expected pets into the [Florida Everglades](https://en.wikipedia.org/wiki/Burmese_pythons_in_Florida).

***

**Exercise**:  Solve $\frac{dy}{dt} = ay(t)$ with $y(0) = y_0$.

*Solution*: Rearrange the equation to be 

$$
  \frac{1}{y}dy = a dt \, ,
$$

and then integrate to obtain

$$
   \ln(y) = a t + C' \, .
$$

Here, $C'$ is an arbitrary integration constant.  Exponentiation of both sides results in

$$
  y(t) = e^{at + C'} = C e^{at} \, ,
$$

where $C = e^{C'}$ is still an arbitrary constant. 

So far, this is just *integral calculus*.  To solve the given IVP, we need to *apply the initial condition* (IC).  Specifically, we need to specify $C$ that satisfies $y(0) = C e^{0} = y_0$.  Clearly, $C = y_0$.

***

**Exercise**:  Solve $\frac{dy}{dt} + py(t) = q$ subject to $y(0) = y_0$ where $p$ and $q$ are constant.

*Solution*: This standard problem is solved by use of an *integration factor*.  Sure, there is a formula you can memorize, but I have a bad memory.  What I do remember is that we want to find some integration factor $\mu(t)$ such that

$$
  \frac{d}{dt}[\mu(t) y(t)]  = \mu(t) q\, .
$$

*Why*?  We've turned the original problem into one for which both sides ($\mu q$ and $\frac{d}{dt}[\mu y]$) can be directly integrated  like we did for $y'=ay$!  Let's apply the product rule:

$$
\frac{d}{dt}[\mu(t) y(t)] = \frac{d\mu}{dt} y(t) + \mu(t) \frac{dy}{dt} = \mu(t) q \, .
$$

By comparison to the original equation, we need

$$
   \frac{d\mu}{dt} y(t) + \mu(t) \frac{dy}{dt} = \mu(t) \frac{dy}{dt} + \mu(t) py(t) \, ,
$$

or 

$$
 \frac{d\mu}{dt} =  \mu(t) p \longrightarrow \frac{d\mu}{\mu} = p dt \longrightarrow \ln \mu =  p t 
$$

or

$$
   \mu(t) = e^{pt} \, .
$$

Finally, integrate the IVP from $0$ to $t$, making sure to use the dummy variable $t'$ inside the integrals:

$$
\begin{split}
  \int^{t}_{0} \frac{d}{dt'}[\mu(t') y(t')] dt' &= \int^{t}_{0} \mu(t') q dt' \\
  \int^{t}_{0} \frac{d}{dt'}[e^{pt'} y(t')] dt' &= \int^{t}_{0} e^{pt'} q dt' \\
  e^{pt} y(t) - e^{0}y(0) &= \frac{q}{p} [e^{pt}-e^{0}] \\
  e^{pt} y(t) - y(0) &= \frac{q}{p} [e^{pt}-1] \, .
\end{split}
$$

A bit of rearranging then leads to 

$$
  y(t) = \frac{q}{p} [1 - e^{pt}] + y(0)e^{-pt} \, .
$$

Of course, this is the result some of you might have memorized, but working through it with the goal of being able to *integrate the equation directly* might shed some new light on (or simply remind you of) the origin of the formula.  

***

**Exercise**: Check that the solution found for $y' + py = q$ actually solves the IVP by substitution of the result into the original equation.

***

**Exercise**:  Consider  $v' = -\frac{c}{m} v(t) + g$.  Can you determine the *terminal velocity* (i.e., the velocity obtained when the falling object is no longer accelerating) *without* solving the IVP?  If not, go ahead and solve the IVP.


***